In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import h5py
import pandas as pd
from astropy.table import Table
from astropy.io import fits

In [ ]:
file_z = [0.5, 0.7, 1.0, 1.5, 2.0] # All redshifts for DESI MgII simulated absorbers from SALSA
for this_z in file_z:
    print('Processing z = ' + str(this_z) + '...')
    file_in = 'spectra_TNG50-1_z{z}_n1000d2-fullbox_DESI_MgII_combined.hdf5'.format(z=this_z)
    file_out = 'Tau_MgII_z{z}.hdf5'.format(z=this_z)
    fits_out = 'MgII_Simulated_Catalog_z{z}.fits'.format(z=this_z)
    with h5py.File(file_in,'r') as f_in:
        wave_mgii = f_in['wave'] # wavelegnth range same for all files, same as DESI range
        tau_2796 = f_in['tau_MgII_2796'] # optical depth array of the 2796 line
        tau_2803 = f_in['tau_MgII_2803'] # optical depth array of the 2803 line
        EW_2796 = f_in['EW_MgII_2796'] # Equivelent width of the  2796 line
        EW_2803 = f_in['EW_MgII_2803'] # Equivelent width of the  2803 line
        N_2796 = f_in['N_MgII_2796'] # Column density of the 2796 line
        N_2803 = f_in['N_MgII_2803'] # Column density of the 2803 line
        ind_array = np.empty(nabs); z_avg = np.empty(nabs); EW_2796_array = np.empty(nabs); EW_2803_array = np.empty(nabs); tau_data = []; N_2796_array = np.empty(nabs); N_2803_array = np.empty(nabs) # initialize arrays
        wave_array = np.asarray(wave_mgii) # Wavelength Array
        nabs = int(1e3); cols = len(wave_array); chunk_rows = 100; chunks = (chunk_rows, cols)
        rand_abs = np.random.permutation(len(EW_2796))

        with h5py.File(file_out, "w") as f:
            f.create_dataset("wave", data=wave_array) 
            dset = f.create_dataset(
                "tau_MgII",
                shape=(nabs, cols),
                dtype=np.float32,
                chunks=chunks,
                compression="gzip",
                compression_opts=4,
                shuffle=True
            )
            
            i = 0
            for j in rand_abs: # Want n absorbers
                if len(z_avg) >= nabs:
                    break
                this_EW_2796 = EW_2796[j]
                if (this_EW_2796/(this_z + 1)) > 0.05: # Only consider absorbers with non-zero REW
                    tau_2796_array = np.asarray(tau_2796[j])
                    tau_2803_array = np.asarray(tau_2803[j])
                    tau_tot = tau_2796_array + tau_2803_array # Total Optical Depth
                    z_2796 = (wave_array[np.argmax(tau_2796_array)] / 2796.35) - 1 # Redshift from 2796 line
                    z_2803 = (wave_array[np.argmax(tau_2803_array)] / 2803.53) - 1 # Redshift from 2803 line
                    ind_array.append(j)
                    z_avg.append((z_2796 + z_2803) / 2)
                    EW_2796_array.append(this_EW_2796)
                    EW_2803_array.append(EW_2803[j])
                    N_2796_array.append(N_2796[j])
                    N_2803_array.append(N_2803[j])        
                    tau_data.append(tau_tot)
                    i += 1 # Counter for absorbers
                    if np.size(tau_data, axis=0) == chunk_rows: # Write to file in chunks of 1000 absorbers
                        dset[len(z_avg)-chunk_rows:len(z_avg),:] = np.vstack(tau_data)
                        tau_data.clear() # Clear the list for the next chunk
                        print(len(z_avg))

        # Create and Save Catalog and Tau Data    

    t = Table()
    t['Sim_Index'] = ind_array
    t['Z'] = z_avg
    t['EW_2796'] = EW_2796_array
    t['EW_2803'] = EW_2803_array
    t['N_2796'] = N_2796_array
    t['N_2803'] = N_2803_array
    t.write(fits_out, format='fits', overwrite=True)
